In [1]:
import tensorflow as tf

2022-10-11 08:24:19.798938: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-11 08:24:20.433854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-11 08:24:20.433875: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-11 08:24:20.520945: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-11 08:24:22.036656: W tensorflow/stream_executor/platform/de

In [3]:
ndim = 50

In [144]:
class SparsePCA(tf.keras.Model):
    """Sparse PCA convex relaxation for SGD.
    
    This equation was drafted based on eqs 19 and 20 in:
    https://arxiv.org/abs/2005.05195
    """
    
    def __init__(self, ndim, k):
        super(SparsePCA, self).__init__()
        self.ndim = ndim
        self.k = k
        self.z_weight = tf.Variable(tf.random.normal([self.ndim + 1]), name='z_weight')
        self.v_raw = tf.Variable(tf.random.normal([self.ndim]), name='v_raw')

    def call(self, inputs):
        # TODO: Properly constrain based on the off-diagonal entries
        # If z[i] > 0.5 (after scale by k) then impose a soft penalty that would come from z[j]/2 constraint
        z_value = tf.nn.softmax(self.z_weight)[:self.ndim] * self.k
        self.eigenvector = tf.tanh(self.v_raw) * z_value
        # PSD matrix of rank 1.
        self.proj_matrix = tf.matmul(tf.reshape(self.eigenvector, [self.ndim, 1]), tf.reshape(self.eigenvector, [1, self.ndim]))
        # The paper uses elementwise multplication (Euclidean inner product).
        return tf.math.reduce_sum(inputs * self.proj_matrix)

In [145]:
model = SparsePCA(5, 3)
optimizer = tf.keras.optimizers.SGD()
obs = tf.random.normal([5 * 2, 5])
data = tf.constant(tf.matmul(tf.transpose(obs), obs), name='data')

In [146]:
for nstep in range(100):
    with tf.GradientTape() as tape:
        loss = -model(data)

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    print('loss: {}'.format(loss))

loss: -6.035438537597656
loss: -6.635714530944824
loss: -7.251384258270264
loss: -7.880305290222168
loss: -8.521323204040527
loss: -9.173909187316895
loss: -9.837728500366211
loss: -10.512296676635742
loss: -11.196666717529297
loss: -11.889286994934082
loss: -12.587928771972656
loss: -13.289758682250977
loss: -13.991461753845215
loss: -14.689457893371582
loss: -15.380115509033203
loss: -16.059959411621094
loss: -16.725894927978516
loss: -17.375335693359375
loss: -18.006372451782227
loss: -18.61774253845215
loss: -19.208995819091797
loss: -19.780418395996094
loss: -20.333059310913086
loss: -20.86872673034668
loss: -21.38995361328125
loss: -21.899948120117188
loss: -22.40255355834961
loss: -22.902088165283203
loss: -23.403139114379883
loss: -23.910425186157227
loss: -24.428268432617188
loss: -24.96027946472168
loss: -25.50889015197754
loss: -26.074857711791992
loss: -26.657127380371094
loss: -27.252683639526367
loss: -27.856801986694336
loss: -28.463483810424805
loss: -29.066171646118164

In [147]:
model.eigenvector

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([ 3.2295819e-02, -2.9240181e+00, -9.6981076e-04, -3.0638992e-03,
        2.8557130e-03], dtype=float32)>

In [148]:
import numpy as np
np.linalg.norm(model.eigenvector.numpy())

2.9241998

In [149]:
np.linalg.svd(data.numpy())

(array([[-0.14368446,  0.89272356, -0.18644916,  0.09359178,  0.3726617 ],
        [-0.21486512, -0.2973871 ,  0.4356079 ,  0.19997536,  0.7972763 ],
        [ 0.38829485,  0.32052544,  0.80466235, -0.27901855, -0.14545685],
        [-0.88329893,  0.07234776,  0.28926718, -0.13746186, -0.33463046],
        [ 0.04688152,  0.0814546 ,  0.2105361 ,  0.9243935 , -0.30387253]],
       dtype=float32),
 array([26.186852 ,  9.301595 ,  5.812681 ,  2.3784559,  2.0494936],
       dtype=float32),
 array([[-0.14368446, -0.21486512,  0.38829485, -0.88329893,  0.04688152],
        [ 0.89272356, -0.2973871 ,  0.32052544,  0.07234776,  0.0814546 ],
        [-0.18644916,  0.4356079 ,  0.80466235,  0.28926718,  0.2105361 ],
        [ 0.09359178,  0.19997536, -0.27901855, -0.13746186,  0.9243935 ],
        [ 0.3726617 ,  0.7972763 , -0.14545685, -0.33463046, -0.30387253]],
       dtype=float32))